In [ ]:
import numpy as np
from scipy.integrate import dblquad
import scipy
from timeit import default_timer as timer
import multiprocessing

LAMBDA = 0.5
D_0 = 20e6
D_LENS = 2e6
FOCUS = 7e6
D_i = 1/(1/FOCUS-1/D_0)


def psf(r_i,r_0,D,f,lmbda):
    [x_i,y_i,z_i] = r_i
    [x_0,y_0,z_0] = r_0

    k = 2 * np.pi / lmbda
    #A = (1 / (-z_0 * z_i * lmbda ** 2)) * np.exp(1j * (k / 2 * z_i) * (x_i ** 2 + y_i ** 2)) * np.exp(
    #    1j * (-k / 2 * z_0) * (x_0 ** 2 + y_0 ** 2))

    A = (1 / (-z_0 * z_i * lmbda ** 2))

    K = lambda x,y: A*np.exp(1j*(k/2)*(-1/z_0+1/z_i-1/f) * (x ** 2 + y ** 2)) * np.exp(-1j * k * ((-x_0 / z_0 + x_i / z_i) * x + (-y_0 / z_0 + y_i / z_i) * y))

    K_real = lambda x, y: scipy.real(K(x,y))
    K_imag = lambda x, y: scipy.imag(K(x,y))

    x_min = -D / 2
    x_max = D / 2
    y_min = lambda x: -np.sqrt((D / 2) ** 2 - x ** 2)
    y_max = lambda x: np.sqrt((D / 2) ** 2 - x ** 2)

    h_real = dblquad(lambda y, x: K_real(x, y), x_min, x_max, y_min, y_max)[0]
    h_imag = dblquad(lambda y, x: K_imag(x, y), x_min, x_max, y_min, y_max)[0]

    return h_real + 1j * h_imag

def psf_rot(r_i, r_0, D, f, lmbda, alpha):

    passive_rot = lambda r,alpha : [r[0], np.cos(alpha)*r[1]-np.sin(alpha)*r[2], np.sin(alpha)*r[1]+np.cos(alpha)*r[2]]

    return psf(passive_rot(r_i,alpha),passive_rot(r_0,alpha),D,f,lmbda)

def test(a, alpha):

    d_0 = D_0
    d_i = D_i
    D = D_LENS
    f = FOCUS
    lmbda = LAMBDA

    alpha_r = np.pi * alpha / (180 * 3600)

    x_min = -a/2
    x_max = a/2
    y_min = lambda x: -np.sqrt(x_max**2 - x**2)
    y_max = lambda x: np.sqrt(x_max**2 - x**2)

    K_real = lambda x,y,x_0,y_0: scipy.real(psf_rot([x,y,d_i],[x_0,y_0,-d_0],D,f,lmbda,alpha_r))

    start = timer()
    integral = dblquad(lambda y_0, x_0: K_real(0, 0, x_0, y_0), x_min, x_max, y_min, y_max)[0]
    end = timer()
    print('Время расчета для {} mu : {} сек'.format(a,np.round(end - start)))

    return [end - start, integral]

def test_mp(x):
    return test(x, 0)


if __name__ == '__main__':
    alpha = 0
    a = [5, 10, 15]
    #test_mp = lambda x: test(x, alpha)

    pool = multiprocessing.Pool(processes=3)

    result = pool.map(test_mp,a)

    print(result)